### Imports & Env Setup

In [1]:
%reload_ext autoreload
%autoreload 2

from dotenv import load_dotenv
load_dotenv()

import dspy

from prompt_migrator.benchmarks import llama_mmlu_pro, leaderboard_mmlu_pro

/Users/knrz/Git/meta-llama/prompt-migrator/.venv/lib/python3.10/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)
/Users/knrz/Git/meta-llama/prompt-migrator/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Configuration

In [2]:
NUM_THREADS = 16

FEW_SHOTS = 5

# See https://docs.litellm.ai/docs/providers/vllm for details
TASK_MODEL = dspy.LM(
    model="gpt-4o-mini",
    # base_url: Optional[str] = None, # or api_base ?
    # api_version: Optional[str] = None,
    # api_key: Optional[str] = None,
    # seed: Optional[int] = None,
    # max_tokens: Optional[int] = None,
    # timeout: Optional[Union[float, int]] = None,
)
PROMPT_MODEL = dspy.LM(
    model="gpt-4o",
    # base_url: Optional[str] = None, # or api_base ?
    # api_version: Optional[str] = None,
    # api_key: Optional[str] = None,
    # seed: Optional[int] = None,
    # max_tokens: Optional[int] = None,
    # timeout: Optional[Union[float, int]] = None,
)

dspy.configure(lm=TASK_MODEL)

# replace this with llama_mmlu_pro or whatever
benchmark = leaderboard_mmlu_pro

# Without chain of thought:
# program = dspy.Predict(
#     benchmark.signature("")
# )

# With chain of thought:
program = dspy.ChainOfThought(
    benchmark.signature("") # put your initial system prompt here, or leave blank
)

evaluate = dspy.Evaluate(
    devset=[],
    metric=benchmark.metric,
    num_threads=NUM_THREADS,
    display_progress=True,
    display_table=True,
    return_all_scores=True,
    return_outputs=True,
)

### Load dataset

In [3]:
trainset, valset, testset = benchmark.datasets(
    train_size=0.1,
    validation_size=0.2,
)

len(trainset), len(valset), len(testset)

(1203, 2165, 8664)

### Optimize Subset + Evaluation

In [4]:
subset_size = 20
optimizer = dspy.MIPROv2(
    metric=benchmark.metric,
    auto="light",
    num_threads=NUM_THREADS,
    task_model=TASK_MODEL,
    prompt_model=PROMPT_MODEL,
    max_labeled_demos=FEW_SHOTS,
)

optimized_program = optimizer.compile(
    program,
    trainset=trainset[:subset_size],
    valset=valset[:subset_size],
    requires_permission_to_run=False,
)

2025/01/15 13:50:48 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: False
num_candidates: 5
valset size: 20

2025/01/15 13:50:48 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/15 13:50:48 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/01/15 13:50:48 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=5 sets of demonstrations...


Bootstrapping set 1/5
Bootstrapping set 2/5
Bootstrapping set 3/5


 25%|██▌       | 5/20 [00:27<01:22,  5.53s/it]


Bootstrapped 4 full traces after 5 examples for up to 1 rounds, amounting to 5 attempts.
Bootstrapping set 4/5


 20%|██        | 4/20 [00:12<00:50,  3.17s/it]


...

2025/01/15 13:54:27 INFO dspy.evaluate.evaluate: Average Metric: 13 / 20 (65.0%)
2025/01/15 13:54:27 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 65.0 with parameters ['Predictor 0: Instruction 4', 'Predictor 0: Few-Shot Set 3'].
2025/01/15 13:54:27 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [0.0, 55.0, 65.0, 55.0, 65.0, 65.0]
2025/01/15 13:54:27 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 65.0
2025/01/15 13:54:27 INFO dspy.teleprompt.mipro_optimizer_v2: =======================


2025/01/15 13:54:27 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 6 / 7 =====



Average Metric: 14.00 / 20 (70.0%): 100%|██████████| 20/20 [00:10<00:00,  1.85it/s]

2025/01/15 13:54:38 INFO dspy.evaluate.evaluate: Average Metric: 14 / 20 (70.0%)
2025/01/15 13:54:38 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far! Score: 70.0
2025/01/15 13:54:38 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 70.0 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 1'].
2025/01/15 13:54:38 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [0.0, 55.0, 65.0, 55.0, 65.0, 65.0, 70.0]
2025/01/15 13:54:38 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 70.0
2025/01/15 13:54:38 INFO dspy.teleprompt.mipro_optimizer_v2: =======================


2025/01/15 13:54:38 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 7 / 7 =====



Average Metric: 11.00 / 20 (55.0%): 100%|██████████| 20/20 [00:13<00:00,  1.46it/s]

2025/01/15 13:54:52 INFO dspy.evaluate.evaluate: Average Metric: 11 / 20 (55.0%)
2025/01/15 13:54:52 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 55.0 with parameters ['Predictor 0: Instruction 4', 'Predictor 0: Few-Shot Set 4'].
2025/01/15 13:54:52 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [0.0, 55.0, 65.0, 55.0, 65.0, 65.0, 70.0, 55.0]
2025/01/15 13:54:52 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 70.0
2025/01/15 13:54:52 INFO dspy.teleprompt.mipro_optimizer_v2: =======================


2025/01/15 13:54:52 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 70.0!


In [13]:
print("BEST PROMPT:\n", optimized_program.predict.signature.instructions)

BEST PROMPT:
 Given the fields `question`, `options`, produce the fields `reasoning`, `answer`.


In [5]:
score, results, all_scores = evaluate(
    optimized_program,
    devset=testset[:subset_size],
)

Average Metric: 11.00 / 20 (55.0%): 100%|██████████| 20/20 [00:12<00:00,  1.65it/s]

2025/01/15 13:55:52 INFO dspy.evaluate.evaluate: Average Metric: 11 / 20 (55.0%)


,question,options,example_answer,reasoning,pred_answer,metric
0,In 1935 roughly how many Americans were in favor of Social Securit...,"[A. 20%, B. 30%, C. 60%, D. 100%, E. 40%, F. 10%, G. 80%, H. 90%, ...",H,Not supplied for this particular example.,C,
1,A circular opening of radius 0.8mm in an opaque screen is illumina...,"['A. Wavelength of light for bright spot: 1.25 × 10^-6 m, Waveleng...",B,"To find the wavelength of light for both bright and dark spots, we...","B. Wavelength of light for bright spot: 1.17 × 10^-6 m, Wavelength...",
2,"For the dissociation reaction cl_2(g) > 2Cl(g) at 1200 °K, calcula...","['A. 1.10 × 10^-4', 'B. 6.89 × 10^-6', 'C. 8.97 × 10^-5', 'D. 1.23...",G,To calculate the equilibrium constant \( K_p \) for the dissociati...,B,
3,With what force does the Earth attract the moon?,"['A. 1.2 × 10^25 dynes', 'B. 5.0 × 10^25 dynes', 'C. 3.0 × 10^25 d...",D,The force with which the Earth attracts the Moon can be calculated...,D,✔️ [True]
4,A beam of electrons has speed 10^7 m/s. It is desired to use the m...,"['A. 0.1 m', 'B. 1 mm', 'C. 1 μm', 'D. 0.01 m', 'E. 1 m', 'F. 10 m...",E,To find the radius of the circle in which the electron beam will t...,E,✔️ [True]
5,Calculate the roost probable distribution and the thermodynamicpro...,['A. most probable distribution is number 3; \\Omega = 250; entrop...,I,Not supplied for this particular example.,B,
6,A cylinder with a movable piston contains a gas at pressure P = 1 ...,"['A. 6 × 10^5 Pa', 'B. 9 × 10^5 Pa', 'C. 3 × 10^5 Pa', 'D. 5 × 10^...",H,"According to Boyle's Law, for a given amount of gas at constant te...",H,✔️ [True]
7,If current real GDP is $5000 and full employment real GDP is at $4...,['A. A decrease in taxes and buying bonds in an open market operat...,F,The current real GDP of $5000 is above the full employment real GD...,G,
8,Describe the function of the lateral-line system in fishes.,['A. The lateral-line system in fishes helps with the digestion of...,G,The lateral-line system in fishes is a specialized sensory system ...,G,✔️ [True]
9,"In class, John's teacher tells him that she will give him the coin...","['A. sensory memory decay', 'B. retroactive interference', 'C. fai...",C,"John's inability to identify the pictures on the coins and bills, ...",C,✔️ [True]


## Medium Optimization

In [ ]:
subset_size = 500
optimizer = dspy.MIPROv2(
    metric=benchmark.metric,
    auto="medium",
    num_threads=NUM_THREADS,
    task_model=TASK_MODEL,
    prompt_model=PROMPT_MODEL,
    max_labeled_demos=FEW_SHOTS,
)

optimized_program = optimizer.compile(
    program,
    trainset=trainset[:subset_size],
    valset=valset[:subset_size],
    requires_permission_to_run=False,
)

In [ ]:
print("BEST PROMPT:\n", optimized_program.predict.signature.instructions)

In [ ]:
score, results, all_scores = evaluate(
    optimized_program,
    devset=testset[:subset_size],
    display_table=False,
)

## Heavy Optimization

In [ ]:
optimizer = dspy.MIPROv2(
    metric=benchmark.metric,
    auto="heavy",
    num_threads=NUM_THREADS,
    task_model=TASK_MODEL,
    prompt_model=PROMPT_MODEL,
    max_labeled_demos=FEW_SHOTS,
)

optimized_program = optimizer.compile(
    program,
    trainset=trainset,
    valset=valset,
)

In [ ]:
print("BEST PROMPT:\n", optimized_program.predict.signature.instructions)

In [ ]:
score, results, all_scores = evaluate(
    optimized_program,
    devset=testset,
    display_table=False,
)